In [8]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import decode_1d

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using {}".format(device))

using cuda


In [7]:
line_phases = np.linspace(start=-np.pi, stop=np.pi, num=200)
line_points = np.zeros((len(line_phases), 2))
line_points[:, 0] = line_phases

In [14]:
line_encoder, line_decoder, line_costs = decode_1d.train(line_points, 0, 1, device, n_training_iterations=1500,
                                                          verbose=True, integration_resamples=30)
print(line_costs)

iteration: 0, decoding loss: 1.6627464294433594, distance cost: 0.0014985838206484914, order reduction: 0.07612977176904678
iteration: 16, decoding loss: 0.9793140888214111, distance cost: 0.3440300524234772, order reduction: 0.062506303191185
iteration: 19, decoding loss: 1.188212513923645, distance cost: 0.1502058506011963, order reduction: 0.02149992436170578
iteration: 25, decoding loss: 1.0661250352859497, distance cost: 0.10012292861938477, order reduction: 0.008631045930087566
iteration: 26, decoding loss: 0.9026782512664795, distance cost: 0.11539090424776077, order reduction: 0.027406316250562668
iteration: 30, decoding loss: 0.6684779524803162, distance cost: 0.15661105513572693, order reduction: 0.18931779265403748
iteration: 31, decoding loss: 0.5805450081825256, distance cost: 0.08133168518543243, order reduction: 0.053802087903022766
iteration: 37, decoding loss: 0.6293122172355652, distance cost: 0.0120597118511796, order reduction: 0.07089436799287796
iteration: 39, dec

In [15]:
ring_encoder, ring_decoder, ring_costs = decode_1d.train(line_points, 1, 0, device, n_training_iterations=1500,
                                                          verbose=True, integration_resamples=30)
print(ring_costs)

iteration: 0, decoding loss: 1.6550676822662354, distance cost: 0.49809741973876953, order reduction: 0.0016625752905383706
iteration: 4, decoding loss: 1.2231171131134033, distance cost: 0.39548322558403015, order reduction: 0.0009572719573043287
iteration: 6, decoding loss: 0.7209251523017883, distance cost: 0.25196513533592224, order reduction: 0.0010167787550017238
iteration: 12, decoding loss: 0.4831049144268036, distance cost: 0.1685267984867096, order reduction: 0.003497066907584667
iteration: 13, decoding loss: 0.2935255765914917, distance cost: 0.2268807590007782, order reduction: 0.0038263073656708
iteration: 15, decoding loss: 0.28270217776298523, distance cost: 0.12442102283239365, order reduction: 0.00101851939689368
iteration: 16, decoding loss: 0.16613347828388214, distance cost: 0.13647356629371643, order reduction: 0.00040018942672759295
iteration: 17, decoding loss: 0.1647130250930786, distance cost: 0.12490878254175186, order reduction: 0.0002603985776659101
iteratio

In [17]:
    with torch.no_grad():
        line_test_embedding = line_encoder(torch.tensor(np.expand_dims(line_phases, -1), dtype=torch.get_default_dtype()).to(device))
        ring_test_embedding = ring_encoder(torch.tensor(np.expand_dims(line_phases, -1), dtype=torch.get_default_dtype()).to(device))

    line_test_embedding = line_test_embedding.cpu().numpy()
    ring_test_embedding = ring_test_embedding.cpu().numpy()

In [ ]:
fig, axs = plt.subplots()
axs.scatter(line_points[:, 0], line_points[:, 1])
axs.plot(line_test_embedding[])